# Setup

In [6]:
import os
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.runnables import chain as as_runnable
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Optional

In [2]:
# Set API key
api_key = os.environ["ANTHROPIC_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]

# Initialize models. We'll use a smaller LLM for most of the work, but a large LLM to distill conversations and writing the final report
fast_llm = ChatOpenAI(api_key=openai_api_key, model="gpt-3.5-turbo")
slow_llm = ChatAnthropic(api_key=api_key, model="claude-3-5-sonnet-20240620")

# Generating the Outline

## Generate Conversations and Resources

In this section we will fetch related articles and develop a set of personas based on the outlines of the fetched related articles. Once the personas have been created, we will then pass them one by one into a conversation cycle.

In the conversation cycle, the persona will ask questions about the topic to an expert on the topic. The expert will take their questions, split it into search queries, then find resources online based on these queries. The expert will then sift through these, collect the trusted sources, and answer the question using these sources. The persona will read the response and ask another question until they have no more questions. The conversation will be saved for the creation of the final outline.

Once all of the personas have conversed with the expert, we will have compiled a set of conversations and trusted resources to be used in the final outline.

This section will be the bulk of the work.

### Fetch Related Articles

In [3]:
class RelatedTopics(BaseModel):
    topics: List[str] = Field(
        description="List of related topics to help in generating personas.",
    )

In [4]:
related_topics_prompt = ChatPromptTemplate.from_template(
"""I'm writing a Wikipedia page for the topic mentioned below. 
Please identify and recommend some related subjects that might be interesting. 
I'm looking for related subjects that provide insights into interesting aspects commonly associated with this topic.

Feel free to list things that are only tangentially related

Please list as many subjects as you can.

Topic of interest: {topic}

make sure to call the RelatedTopics function.
"""
)

related_topics_chain = related_topics_prompt | fast_llm.with_structured_output(
    RelatedTopics
)

In [5]:
example_topic = "AI agents and the potential economic, social, and environmental impacts."
related_topics = await related_topics_chain.ainvoke({"topic": example_topic})
related_topics

RelatedTopics(topics=['Artificial Intelligence', 'Machine Learning', 'Automation', 'Robotics', 'Ethics in AI', 'Job Displacement', 'Universal Basic Income', 'Data Privacy', 'Algorithm Bias', 'Smart Cities', 'Climate Change', 'Sustainable Development', 'Future of Work', 'Digital Transformation', 'Human-Machine Interaction', 'AI Ethics', 'Social Implications of AI', 'Impact of AI on Healthcare', 'Environmental Sustainability', 'AI in Finance'])

In [ ]:
# Get related articles with wikipedia retriever
wikipedia_retriever = WikipediaRetriever(load_all_available_meta=True, top_k_results=1)

### Create Personas

### Converse With Expert

## Generate Draft Outline

The draft outline is a rough draft of the outline based only on the task. The collected conversations and resources from above will then be used to refine the draft outline.

## Generate Final Outline

Use the collected conversations and resources to refine the draft outline. The result will be the final outline.

# Generate the Final Article

Each section will be generated independently using the previously gathered resources. Once each section is generated, we will have a large model go over the entire article to ensure consistency and remove duplicate information.